### Task 2

#### Task 2.1

In [1]:
# Load data (csv file)
data <- read.csv("transformer_data.csv", header = TRUE, sep = ",")
# Display the first few rows of the data
head(data)

,time,Y,Ta,S,I
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,23.49673,7.083333,0,15.89954
2,2,22.72033,7.116667,0,15.86177
3,3,22.05270,7.216667,0,16.12147
4,4,21.53805,7.250000,0,15.44763
5,5,20.99503,7.316667,0,16.25832
6,6,20.61153,7.483333,0,16.63265


In [2]:
# Plot data

png("2_zoom_out_plot_dual_axis.png", width = 12, height = 6, units = "in", res = 300)

# Plot main variables (temperature in °C)
par(bg = "white", mar = c(5, 4, 4, 4) + 0.1)  # Leave space on right for second y-axis
plot(data$Y, type = "l", col = "blue", xlab = "Time (h)", ylab = "Temperature (°C)", 
     main = "Transformer Data with Solar Radiance", 
     ylim = range(c(data$Y, data$Ta, data$I)), lwd = 2)
lines(data$Ta, col = "springgreen2", lwd = 2)
lines(data$I, col = "magenta2", lwd = 2)

# Add additional y-axis label for "Load" in pink
mtext("Load (kA)", side = 2, line = 2, col = "magenta2", at = mean(par("usr")[3:4]))

# Add solar radiance (S) on a secondary axis
par(new = TRUE)
plot(data$S, type = "l", col = "goldenrod2", axes = FALSE, xlab = "", ylab = "", lwd = 2)
axis(side = 4, col = "goldenrod2", col.axis = "goldenrod2")
mtext("Solar Radiance (W/m²)", side = 4, line = 3, col = "goldenrod2")

# Legend
legend("topright", 
       legend = c("Y (°C)", "Ta (°C)", "I (kA)", "S (W/m²)"), 
       col = c("blue", "springgreen2", "magenta2", "goldenrod2"), 
       lty = 1, lwd = 2)

dev.off()


pdf 
  2

### Task 2.2

In [ ]:
# Functions Exercise 2
kf_logLik_dt <- function(par, df) {
  # par: vector of parameters
  # df: data frame with observations and inputs as columns (Y, Ta, S, I)
  # par: Could be on the form c(A11, A12, A21, A22, B11, B12, B21, B22, Q11, Q12, Q22)
  A   <- matrix(par[1:4]) # transition matrix
  B   <- matrix(par[5:8]) # input matrix
  Qlt <- matrix(par[9:11]) # lower-triangle of system covariance matrix
  Q_mat <- Qlt %*% t(Qlt) # system covariance matrix
  Sigma1lt <- matrix(Q_mat) # lower-triangle of system covariance matrix
  # The system covariance matrix is given by Qlt %*% t(Qlt) and is symmetric positive definite
  Sigma1   <- Sigma1lt %*% t(Sigma1lt)
  C   <- matrix(1) # observation matrix
  Sigma2 <- matrix() # observation noise covariance matrix
  X0  <- matrix(20) # initial state

  # Variables
  obs_cols <- c("Y") # observation column names
  input_cols <- c("Ta","S","I") # input column names

  # pull out data
  Y  <- as.matrix(df[, obs_cols])     # m×T
  U  <- as.matrix(df[, input_cols])   # p×T
  Tn <- nrow(df)

  # init
  n      <- nrow(A)
  x_est  <- matrix(Y[1,], n, 1)            # start state from first obs
  x_est <- X0 
  P_est  <- diag(1e1, n)                   # X0 prior covariance
  logLik <- 0

  for (t in 1:Tn) {
    # prediction step
    # write the prediction step
    x_pred <- A %*% x_est + B %*% matrix(U[t, ], ncol = 1)
    # write the prediction step (Sigma_xx)
    P_pred <- A %*% P_est %*% t(A) + Sigma1 # Variance of prediction

    # innovation step
    y_pred  <- C %*% x_est                                    # predicted observation
    S_t     <- C %*% P_est %*% t(C) + Sigma2                           # predicted observation covariance (Sigma_yy)
    innov   <- Y[t, ] - y_pred                                # innovation (one-step prediction error)

    # log-likelihood contribution
    logLik <- logLik - 0.5*(sum(log(2*pi*S_t)) + t(innov) %*% solve(S_t, innov))

    # update step
    K_t    <- P_pred[t] / innov[t] # Kalman gain
    x_est  <- x_pred[t] + K_t * innov[t] # reconstructed state
    P_est  <- P_pred[t] - K_t * P_pred[t] # reconstructed covariance
  }

  as.numeric(logLik)
}

In [18]:
# Optimizer wrapper
estimate_dt <- function(start_par, df, lower=NULL, upper=NULL) {
  negLL <- function(par){ -kf_logLik_dt(par, df) }
  optim(
    par    = start_par, fn = negLL,
    method = "L-BFGS-B",
    lower  = lower, upper = upper,
    control= list(maxit=1000, trace=1)
  )
}

In [19]:
# Setting starting parameter values
start_par <- c(
  0.1, 0.2, 0.3, 0.4, # A
  0.5, 0.6, 0.7, 0.8, # B
  0.9, 1.0, 1.1      # Q
)

lower_bound <- c(
  0.0, 0.0, 0.0, 0.0, # A
  0.0, 0.0, 0.0, 0.0, # B
  1e-3, 1e-3, 1e-3   # Q
)

upper_bound <- c(
  1.0, 1.0, 1.0, 1.0, # A
  1.0, 1.0, 1.0, 1.0, # B
  1e3, 1e3, 1e3      # Q
)

In [ ]:
kf_logLik_dt2 <- function(par, df) {
  # par: vector of parameters
  # df: data frame with observations and inputs as columns (Y, Ta, S, I)
  # par: c(A11, A12, A21, A22, B11, B12, B21, B22, Q, R, x0)
  # A: 2x2, B: 2x3, Q: scalar, R: scalar, x0: scalar

  # Define dimensions
  n <- 2  # state dimension
  m <- 1  # observation dimension
  p <- 3  # input dimension

  # Extract parameters
  A <- matrix(par[1:4], nrow = n, ncol = n, byrow = TRUE)
  B <- matrix(par[5:10], nrow = n, ncol = p, byrow = TRUE)
  Q <- diag(abs(par[11]), n)  # process noise covariance (positive)
  R <- matrix(abs(par[12]), m, m)  # observation noise covariance (positive)
  C <- matrix(c(1, 0), nrow = m, ncol = n)  # observation matrix (fixed)
  x0 <- matrix(par[13], nrow = n, ncol = 1)  # initial state

  # Variables
  obs_cols <- c("Y") # observation column names
  input_cols <- c("Ta","S","I") # input column names

  # pull out data
  Y  <- as.matrix(df[, obs_cols])     # T x m
  U  <- as.matrix(df[, input_cols])   # T x p
  Tn <- nrow(df)

  # init
  x_est  <- x0
  P_est  <- diag(1e1, n)
  logLik <- 0

  for (t in 1:Tn) {
    # prediction step
    x_pred <- A %*% x_est + B %*% U[t, ]
    P_pred <- A %*% P_est %*% t(A) + Q

    # innovation step
    y_pred  <- C %*% x_pred
    S_t     <- C %*% P_pred %*% t(C) + R
    innov   <- Y[t, ] - y_pred

    # log-likelihood contribution
    logLik <- logLik - 0.5 * (log(det(2 * pi * S_t)) + t(innov) %*% solve(S_t, innov))

    # update step
    K_t    <- P_pred %*% t(C) %*% solve(S_t)
    x_est  <- x_pred + K_t %*% innov
    P_est  <- (diag(n) - K_t %*% C) %*% P_pred
  }

  as.numeric(logLik)
}

In [21]:
# Run function

kf_logLik_dt(start_par, data)
estimate_dt(start_par, data, lower_bound, upper_bound)

ERROR: Error in B %*% matrix(U[t, ], ncol = 1): non-conformable arguments
